In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split

In [11]:
ecg_feature = pd.read_csv('./ecg_features.csv')
ppg_feature = pd.read_csv('./ppg_features.csv')
eda_feature = pd.read_csv('./eda_features.csv')
feel_feature = pd.read_csv('./feeling_labels.csv')
id_feature = pd.read_csv('./id_labels.csv')

In [12]:
id_feature

,Unnamed: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
1,1,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
2,2,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
3,3,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
4,4,0.007489,-0.002914,0.012315,0.016057,0.007052,0.000804,0.01158,0.003371,0.005506,-0.006168,-0.007888,-0.009054,0.008220,-0.000470,0.002197,-0.008175,0.013287,-0.002640,-0.010925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,229,0.003141,-0.006964,0.010231,0.005548,0.012103,0.000373,0.00070,-0.001923,-0.003356,-0.005007,-0.003531,-0.002360,0.002836,-0.001696,-0.006686,-0.007511,0.015053,0.009562,-0.020179
230,230,0.003141,-0.006964,0.010231,0.005548,0.012103,0.000373,0.00070,-0.001923,-0.003356,-0.005007,-0.003531,-0.002360,0.002836,-0.001696,-0.006686,-0.007511,0.015053,0.009562,-0.020179
231,231,0.003141,-0.006964,0.010231,0.005548,0.012103,0.000373,0.00070,-0.001923,-0.003356,-0.005007,-0.003531,-0.002360,0.002836,-0.001696,-0.006686,-0.007511,0.015053,0.009562,-0.020179
232,232,0.003141,-0.006964,0.010231,0.005548,0.012103,0.000373,0.00070,-0.001923,-0.003356,-0.005007,-0.003531,-0.002360,0.002836,-0.001696,-0.006686,-0.007511,0.015053,0.009562,-0.020179


In [13]:
ecg_feature = ecg_feature.drop(['Unnamed: 0'], axis = 1)
ppg_feature = ppg_feature.drop(['Unnamed: 0'], axis = 1)

In [14]:
X_train = pd.concat([ecg_feature, ppg_feature, eda_feature], axis = 1).values

In [15]:
label_data = pd.read_csv('./train_label_softmax.csv')
y_train = label_data.values

In [7]:
X_train = np.expand_dims(X_train, axis = 0)

In [16]:
X_train

array([[12.140015, 19.875168,  0.      , ..., 21.984102, 21.86208 ,
        13.78929 ],
       [13.15213 , 21.535746,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [13.004108, 21.283129,  0.      , ..., 24.902727, 23.910677,
        16.990757],
       ...,
       [11.715818, 19.185234,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [12.60052 , 20.63363 ,  0.      , ...,  0.      , 22.506584,
         0.      ],
       [10.416733, 17.057709,  0.      , ..., 30.521257, 30.937357,
         0.      ]])

In [18]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(Flatten())

batch_size = 1
num_samples = 246
features = []

for i in range(0, num_samples, batch_size):
    batch = X_train[i:i+batch_size]
    extracted_features = model.predict(batch)
    features.append(extracted_features)

1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 17ms/step


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [26]:
feat_df = pd.DataFrame(np.array(features).reshape(234, 64))
feat_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.0,0.0,0.0,0.0,1.297414,0.0,0.000000,0.0,0.0,0.451785,...,0.917846,0.0,0.041375,0.982751,0.0,0.0,0.0,0.293124,0.0,1.071035
1,0.0,0.0,0.0,0.0,1.256827,0.0,0.004872,0.0,0.0,0.425848,...,0.897308,0.0,0.011597,0.951302,0.0,0.0,0.0,0.273451,0.0,1.059863
2,0.0,0.0,0.0,0.0,1.412122,0.0,0.000000,0.0,0.0,0.510493,...,1.012034,0.0,0.026823,1.070510,0.0,0.0,0.0,0.317143,0.0,1.193807
3,0.0,0.0,0.0,0.0,0.985730,0.0,0.000000,0.0,0.0,0.350418,...,0.705529,0.0,0.027102,0.753385,0.0,0.0,0.0,0.219282,0.0,0.823645
4,0.0,0.0,0.0,0.0,1.123323,0.0,0.019762,0.0,0.0,0.361714,...,0.774484,0.0,0.038965,0.870805,0.0,0.0,0.0,0.271497,0.0,0.889309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,0.0,0.0,0.0,0.0,1.546806,0.0,0.000000,0.0,0.0,0.532108,...,1.122261,0.0,0.130725,1.155014,0.0,0.0,0.0,0.313745,0.0,1.310231
230,0.0,0.0,0.0,0.0,1.569289,0.0,0.000000,0.0,0.0,0.583760,...,1.151189,0.0,0.063122,1.185126,0.0,0.0,0.0,0.277497,0.0,1.332973
231,0.0,0.0,0.0,0.0,1.704716,0.0,0.000000,0.0,0.0,0.570607,...,1.235755,0.0,0.086471,1.331751,0.0,0.0,0.0,0.330371,0.0,1.408774
232,0.0,0.0,0.0,0.0,1.743222,0.0,0.000000,0.0,0.0,0.615882,...,1.230851,0.0,0.072251,1.331031,0.0,0.0,0.0,0.281807,0.0,1.508834


In [28]:
X_train = pd.concat([feat_df, feel_feature, id_feature], axis = 1).values

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy', metrics=['accuracy'])
#model.add(Flatten())
model.fit(X_train, y_train, epochs=500, batch_size=32)

In [ ]:
output_layer = Dense(6, activation='softmax')(x)  

model = Model(inputs=input_layer, outputs=output_layer)